## Moat Brand Reporting Refresh

In [1]:
from rtf_utils.moat_utils import MoatTile
from essence.analytics.platform import securedcredentials as secure_creds
from rtf_utils.gcp_utils import BigQuery,CloudStorage
import warnings

warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")
import json

In [2]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)

moat_token = secure_creds.getDataFromEssenceVault('Moat_Token_Google')

In [3]:
start_date = '20190701'
end_date = '20190917'

In [4]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

In [21]:
bonito = [(13120,{'level1':10154328017481183},['date','level4']),
          (13386,{'level1':7020493427},['date','level4'])]

In [67]:
def get_moat_data(tile_id,start_date, end_date,level_filters=None,dimensions=None,**kwargs):
    tile = MoatTile(tile_id=tile_id,level_filters=level_filters,dimensions=dimensions)
    local_filename = tile.get_data(start_date,end_date,moat_token)
    return local_filename

def upload_local_file(local_filename,dest_bucket,folder=None):
    gcs = CloudStorage(credentialsFromVault)
    if folder:
        blob_name = folder + "/" + local_filename
    else:
        blob_name = local_filename
    gcs.upload_blob(dest_bucket, blob_name,local_filename,mode='filename')
    print("File Uploaded")
    return "gs://" + dest_bucket + "/" + blob_name

In [69]:
from rtf_utils.bq_schema import moat_schema_dict

# Prototype WorkFlow

In [71]:
for x in bonito:
    tile_id, level_filter,dimensions = x
    
    filename = get_moat_data(tile_id,start_date,end_date,level_filter,dimensions)
    print("Stored at {}".format(filename))    
    file_uri = upload_local_file(filename,"rtf_staging",folder="brand_reporting")
    
    print("Trying Schema")
    schema_dict = moat_schema_dict[tile_id]
    
    dest_table = "{}_{}".format(schema_dict['bq_name'],tile_id)
    
    print("try upload")
    
    resp = bq.load_from_gcs("rtf_brand_reporting",
                     file_uri,
                    dest_table,
                    schema=schema_dict['schema'],
                    extension='json')
    
    #add cleaning step
    print(resp['jobReference']) 

Data Available, Storing
Stored at 13120_V_instagram.json
File Uploaded
Trying Schema
try upload
{'_retry': <google.api_core.retry.Retry object at 0x10ffae1d0>, '_result': None, '_exception': None, '_result_set': False, '_polling_thread': None, '_done_callbacks': [], '_properties': {'kind': 'bigquery#job', 'etag': 'v4UlvvyvzjZVKwueO1LFpw==', 'id': 'essence-analytics-dwh:US.e17ee46a-ec11-4ac5-b91f-b6b947f1a979', 'selfLink': 'https://www.googleapis.com/bigquery/v2/projects/essence-analytics-dwh/jobs/e17ee46a-ec11-4ac5-b91f-b6b947f1a979?location=US', 'user_email': '131786951123-compute@developer.gserviceaccount.com', 'configuration': {'load': {'sourceUris': ['gs://rtf_staging/brand_reporting/13120_V_instagram.json'], 'schema': {'fields': [{'name': 'date', 'type': 'DATE', 'mode': 'REQUIRED'}, {'name': 'level1_id', 'type': 'INTEGER', 'mode': 'REQUIRED'}, {'name': 'level1_label', 'type': 'STRING', 'mode': 'REQUIRED'}, {'name': 'level4_id', 'type': 'INTEGER', 'mode': 'REQUIRED'}, {'name': 'lev

In [59]:
from rtf_utils.gcp_utils import BigQuery